# 🐰 Easter Bunny Optimised Planning Tool 🐰


### Install Packages

In [39]:
# %pip install -r requirements.txt

### Setup Packages 


In [40]:
from ortools.linear_solver import pywraplp

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from typing import Dict, List
from dataclasses import dataclass, field

### 🐰 Basic Unconstrained Model: Maximizing Egg Collection
Before adding constraints, we start with a simple model where the objective is to maximize the number of eggs collected.

At this stage, all children collect as many eggs as possible without constraints.

In [58]:
### Import Solver
# Create the solver
solver = pywraplp.Solver.CreateSolver('SCIP')

### Declare Variables
# Define the number of eggs per child (unconstrained)
num_children = 5
num_eggs = [15, 12, 20, 10, 18]  # Eggs each child collects

# Define variables (x[i] represents number of eggs child i collects)
x = [solver.IntVar(0, eggs, f"child_{i}") for i, eggs in enumerate(num_eggs)]

### Define Objective
# Objective: Maximize total eggs collected
solver.Maximize(solver.Sum(x))

### Define Constraints
## // no constraints 

### Solve Problem 
status = solver.Solve()

### Output
if status == pywraplp.Solver.OPTIMAL:
    print("Optimal Solution Found:")
    for i in range(num_children):
        print(f'Child {i+1}: {x[i].solution_value()} eggs')
else:
    print("No optimal solution found.")

Optimal Solution Found:
Child 1: 15.0 eggs
Child 2: 12.0 eggs
Child 3: 20.0 eggs
Child 4: 10.0 eggs
Child 5: 18.0 eggs


## Constraints 

### 🧺 Basket Capacity (Knapsack Constraint)


$$
\sum_{j=1}^{M} x_{ij} \leq b_i \quad \forall i
$$

Each child now has a limited basket size, meaning they can only carry a certain number of eggs.

Now, each child must choose eggs within their basket limit.

In [42]:
### Import Solver
# # Create the solver
solver = pywraplp.Solver.CreateSolver('SCIP')

### Declare Variables
# Variables
num_children = 5
num_eggs = [15, 12, 20, 10, 18] 

## NEW Variable 
# Basket capacity per child
basket_capacity = [10, 8, 12, 6, 10]

# Define variables (x[i] represents number of eggs child i collects)
x = [solver.IntVar(0, eggs, f'child_{i}') for i, eggs in enumerate(num_eggs)]

### Define Objective
# Objective: Maximize total eggs collected
solver.Maximize(solver.Sum(x))

### Define Constraints
# Add basket capacity constraints
for i in range(num_children): ## For each child
    ## Ensure x[i] (number of eggs) < capacity
    solver.Add(x[i] <= basket_capacity[i]) 

### Solve Problem 
# Solve again
status = solver.Solve()


### Output
if status == pywraplp.Solver.OPTIMAL:
    print("Optimal Solution Found:")
    for i in range(num_children):
        print(f'Child {i+1}: {x[i].solution_value()} eggs')
else:
    print("No optimal solution found.")


Optimal Solution Found:
Child 1: 10.0 eggs
Child 2: 8.0 eggs
Child 3: 12.0 eggs
Child 4: 6.0 eggs
Child 5: 10.0 eggs


### ⏱️ Adding a Time Constraint

$$
\sum_{j=1}^{M} x_{ij} \leq r \cdot \frac{t_i}{d_i} \quad \forall i
$$

Now, we introduce a time constraint, where each child can only collect for a fixed duration.

Now, each child is limited by how quickly they can collect eggs within their time frame.

In [43]:
### Import Solver
# # Create the solver
solver = pywraplp.Solver.CreateSolver('SCIP')

### Declare Variables
num_children = 5
num_eggs = [15, 12, 20, 10, 18] 
basket_capacity = [10, 8, 12, 6, 10]

## NEW VARIABLES:
time_limit = [5, 4, 6, 3, 5]  # Each child has different time constraints
egg_collection_rate = 3  # Eggs per minute

# Define variables (x[i] represents number of eggs child i collects)
x = [solver.IntVar(0, eggs, f'child_{i}') for i, eggs in enumerate(num_eggs)]

### Define Objective
# Objective: Maximize total eggs collected
solver.Maximize(solver.Sum(x))

### Define Constraints
#Constraint: Basket capacity 
for i in range(num_children): 
    solver.Add(x[i] <= basket_capacity[i]) 

## NEW CONSTRAINT
# Add time constraints to the model
for i in range(num_children): 
    # ensure number of eggs < time x number of eggs per minute
    solver.Add(x[i] <= time_limit[i] * egg_collection_rate)

### Solve again
status = solver.Solve()

# Output
if status == pywraplp.Solver.OPTIMAL:
    print("Optimal Solution Found:")
    for i in range(num_children):
        print(f'Child {i+1}: {x[i].solution_value()} eggs')
else:
    print("No optimal solution found.")


Optimal Solution Found:
Child 1: 10.0 eggs
Child 2: 8.0 eggs
Child 3: 12.0 eggs
Child 4: 6.0 eggs
Child 5: 10.0 eggs


### 🥚 Different Egg Types & Values (Weighted Knapsack)


Eggs now have different values:

Regular eggs: 1 point

Golden eggs: 5 points

Healthy eggs: 2 points

Chocolate eggs: 3 points

Children prioritise high-value eggs within their basket capacity.

Now, each child picks eggs based on value rather than just quantity.

In [ ]:
# Define variables
num_children = 5
num_eggs = [15, 12, 20, 10, 18] 
basket_capacity = [10, 8, 12, 6, 10]

## NEW VARIABLES:
time_limit = [5, 4, 6, 3, 5]  # Each child has different time constraints
egg_collection_rate = 3  # Eggs per minute

# Define egg types and their respective values
egg_values = [1, 5, 2, 3]  # Regular, Golden, Healthy, Chocolate

# Define number of each egg type available per child
### Note - this is done for simplicity, we will more to a single supply of eggs later. 
egg_counts = [
    [5, 2, 4, 4],  # Child 1
    [3, 3, 2, 2],  # Child 2
    [6, 1, 3, 2],  # Child 3
    [4, 2, 1, 3],  # Child 4
    [5, 2, 2, 3]   # Child 5
]

# Create new decision variables for each egg type
x = [[solver.IntVar(0, egg_counts[i][j], f'child_{i}_eggtype_{j}')
      for j in range(4)] for i in range(num_children)]

## NEW OBJECTIVE FUNCTION
# Update objective to maximize total egg value
solver.Maximize(sum(x[i][j] * egg_values[j] for i in range(num_children) for j in range(4)))

# Basket capacity & time constraints
for i in range(num_children):
    solver.Add(sum(x[i]) <= basket_capacity[i])
    solver.Add(sum(x[i]) <= time_limit[i] * egg_collection_rate)


# Solve again
status = solver.Solve()

# Output
if status == pywraplp.Solver.OPTIMAL:
    print("Optimal solution found!\n")
    for i in range(num_children):
        print(f'Child {i+1}:')
        total_eggs = 0
        for j in range(len(egg_values)):
            count = int(x[i][j].solution_value())
            print(f"  {egg_values[j]} eggs: {count}")
            total_eggs += count
        print(f"  Total eggs: {total_eggs}\n")
else:
    print("No optimal solution found.")

Optimal solution found!

Child 1:
  1 eggs: 0
  5 eggs: 2
  2 eggs: 4
  3 eggs: 4
  Total eggs: 10

Child 2:
  1 eggs: 1
  5 eggs: 3
  2 eggs: 2
  3 eggs: 2
  Total eggs: 8

Child 3:
  1 eggs: 6
  5 eggs: 1
  2 eggs: 3
  3 eggs: 2
  Total eggs: 12

Child 4:
  1 eggs: 0
  5 eggs: 2
  2 eggs: 1
  3 eggs: 3
  Total eggs: 6

Child 5:
  1 eggs: 3
  5 eggs: 2
  2 eggs: 2
  3 eggs: 3
  Total eggs: 10



### ⚖️ Fair Distribution Constraint

$$
\left| \sum_{j=1}^{M} x_{ij} - \sum_{j=1}^{M} x_{kj} \right| \leq \theta \quad \forall i, k \text{ with } i < k
$$

Ensure that no child gets significantly more eggs than another.

Now, the hunt is fair, preventing one child from dominating (no 6'5" 28 year old getting all the eggs).

In [45]:
# Create the solver
solver = pywraplp.Solver.CreateSolver('SCIP')

# Define variables
num_children = 5
num_eggs = [15, 12, 20, 10, 18] 
time_limit = [5, 4, 6, 3, 5] 
egg_collection_rate = 3  

## NEW PARAMETER
# Define maximum egg collection difference allowed
fairness_threshold = 3


# Create new decision variables for each egg type
x = [[solver.IntVar(0, egg_counts[i][j], f'child_{i}_eggtype_{j}')
      for j in range(4)] for i in range(num_children)]

# Update objective to maximize total egg value
solver.Maximize(sum(x[i][j] * egg_values[j] for i in range(num_children) for j in range(4)))

# Basket capacity constraints
for i in range(num_children):
    solver.Add(sum(x[i]) <= basket_capacity[i])


## NEW CONSTRAINT:
# Fairness - no child should exceed another's egg count by threshold
for i in range(num_children):
    for j in range(i + 1, num_children):
        solver.Add(
            sum(x[i]) - sum(x[j]) <= fairness_threshold
        )
        solver.Add(
            sum(x[j]) - sum(x[i]) <= fairness_threshold
        )

# Solve again
status = solver.Solve()

# Output
if status == pywraplp.Solver.OPTIMAL:
    print("Optimal solution found!\n")
    for i in range(num_children):
        print(f'Child {i+1}:')
        total_eggs = 0
        for j in range(len(egg_values)):
            count = int(x[i][j].solution_value())
            print(f"  {egg_values[j]} eggs: {count}")
            total_eggs += count
        print(f"  Total eggs: {total_eggs}\n")
else:
    print("No optimal solution found.")


Optimal solution found!

Child 1:
  1 eggs: 0
  5 eggs: 2
  2 eggs: 3
  3 eggs: 4
  Total eggs: 9

Child 2:
  1 eggs: 1
  5 eggs: 3
  2 eggs: 2
  3 eggs: 2
  Total eggs: 8

Child 3:
  1 eggs: 3
  5 eggs: 1
  2 eggs: 3
  3 eggs: 2
  Total eggs: 9

Child 4:
  1 eggs: 0
  5 eggs: 2
  2 eggs: 1
  3 eggs: 3
  Total eggs: 6

Child 5:
  1 eggs: 2
  5 eggs: 2
  2 eggs: 2
  3 eggs: 3
  Total eggs: 9



### ⛰️ Terrain & Accessibility Constraints

$$
\sum_{j=1}^{M} x_{ij} \leq r \cdot \frac{t_i}{d_i} \quad \forall i
$$

Now, eggs are placed in different locations, some of which are harder to reach.

Children must optimize not just for value but also for how difficult eggs are to reach.

In [46]:
# Define terrain difficulty multipliers (1 = easy, 2 = medium, 3 = hard)
terrain_difficulty = [1, 2, 3, 1, 2]

# Modify time constraint to include terrain difficulty
for i in range(num_children):
    solver.Add(sum(x[i]) <= time_limit[i] * egg_collection_rate / terrain_difficulty[i])


# Solve again
status = solver.Solve()

# Output
if status == pywraplp.Solver.OPTIMAL:
    print("Optimal solution found!\n")
    for i in range(num_children):
        print(f'Child {i+1}:')
        total_eggs = 0
        for j in range(len(egg_values)):
            count = int(x[i][j].solution_value())
            print(f"  {egg_values[j]} eggs: {count}")
            total_eggs += count
        print(f"  Total eggs: {total_eggs}\n")
else:
    print("No optimal solution found.")


Optimal solution found!

Child 1:
  1 eggs: 0
  5 eggs: 2
  2 eggs: 3
  3 eggs: 4
  Total eggs: 9

Child 2:
  1 eggs: 0
  5 eggs: 3
  2 eggs: 1
  3 eggs: 2
  Total eggs: 6

Child 3:
  1 eggs: 0
  5 eggs: 1
  2 eggs: 3
  3 eggs: 2
  Total eggs: 6

Child 4:
  1 eggs: 0
  5 eggs: 2
  2 eggs: 1
  3 eggs: 3
  Total eggs: 6

Child 5:
  1 eggs: 0
  5 eggs: 2
  2 eggs: 2
  3 eggs: 3
  Total eggs: 7



### 🧱 **Budget or Cost Constraint (Easter Bunny side)**
Each egg type has a cost to hide (golden = expensive), and the Easter Bunny has a total **budget** $ B $:
$$
\sum_{i=1}^{N} \sum_{j=1}^{M} c_j \cdot x_{ij} \leq B
$$  
Where $ c_j $ is the cost of placing egg type $ j $.

💡 **Use case:** Useful when simulating **real-world planning** or limiting resources.



### 🎯 **Prioritization / Personal Preference**
**Constraint:** Some children prefer certain types of eggs (e.g., healthier or chocolatey ones).  
$$
x_{ij} \leq p_{ij} \quad \forall i, j
$$
Where $ p_{ij} $ is the **maximum preference weight** (e.g., Child 3 doesn’t like chocolate, so $ p_{3, \text{chocolate}} = 0 $).

💡 **Use case:** Encourage personalization or diet-based optimization.


### 📈 **Minimum Enjoyment or Health Threshold**
**Constraint:** Ensure each child collects a minimum enjoyment or health value.  
$$
\sum_{j} e_j \cdot x_{ij} \geq E_{\text{min}} \quad \forall i
$$  
Where $ e_j $ is enjoyment score (or health score), and $ E_{\text{min}} $ is a required level.  

💡 **Use case:** Prevents over-optimization for value alone—adds **nutritional or fun balance**.


### 🎉 **Special Egg Distribution Constraint**
**Constraint:** Limit how many **golden eggs** each child can have, or enforce at least one per child.  

$$
x_{i, \text{golden}} \geq 1 
$$

We want each child to find at least one special egg



### 🧠 **Cognitive Load / Decision Complexity**
**Constraint:** Limit the number of different egg types a child can collect.  
$$
\sum_{j=1}^{M} y_{ij} \leq K \quad \forall i
$$  
Where $ y_{ij} \in \{0,1\} $ is a binary indicating if child $ i $ picks egg type $ j $, and $ K $ is the max number of types.

💡 **Use case:** Models that simulate attention, simplicity, or younger kids focusing on fewer egg types.



### 🧭 **Distance-Based Placement Constraint**
**Constraint:** Egg types can only be placed within a specific radius of a child’s start point.  
This turns it into a **spatial optimization problem** (possibly a mixed integer program), where terrain and geography dictate availability.  

💡 **Use case:** Add realism to large-scale hunts in parks or fields.


### 👯 **Buddy Constraint (Team-based Hunting)**
**Constraint:** Some children must stay within a similar egg count or collaborate as a pair.  
$$
| \sum_{j} x_{ij} - \sum_{j} x_{kj} | \leq \delta \quad \text{for buddies } i, k
$$

💡 **Use case:** Useful for modeling **group fairness** (e.g., siblings hunt together).


### 🧠 **Strategic Trade Zones (Exchange or Trading)**
Post-collection, you could even model a **second stage** where children trade eggs to optimize personal preference, health, or fun. That becomes a **multi-stage optimization game**.


# Model with Classes

#### Setup Classes

In [53]:
from dataclasses import dataclass, field
from typing import Dict, List
from ortools.linear_solver import pywraplp


@dataclass
class EggType:
    name: str
    value: int
    enjoyment: int
    health: int
    size: int
    cost: int


@dataclass
class Child:
    name: str
    age: int
    basket_capacity: int
    preference: Dict[str, bool]
    house_id: int

    def effective_time(self, base_time=15, age_handicap_per_year=0.5, age_threshold=10) -> float:
        extra_time = max(0, self.age - age_threshold) * age_handicap_per_year
        return max(0, base_time - extra_time)


@dataclass
class House:
    id: int
    children: List[Child]

@dataclass
class ModelParameters:
    golden_egg_name: str = "golden"
    health_min: int = 5
    happiness_min: int = 5
    egg_rate: int = 3
    fairness_threshold: int = 2


#### Setup Optimiser Function

In [54]:
def optimize_easter_distribution(
    houses: List[House],
    egg_types: Dict[str, EggType],
    total_supply: Dict[str, int],
    model_parameters: ModelParameters
) -> Dict[str, Dict[str, int]]:

    ### Import Solver
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        return {}

    ### Declare Variables
    ## Process data inputs
    all_children = [child for house in houses for child in house.children]
    egg_names = list(egg_types.keys())

    ## Setup decision variables
    x = {}
    for child in all_children:
        for egg in egg_names:
            x[(child.name, egg)] = solver.IntVar(0, total_supply[egg], f'x_{child.name}_{egg}')
    
    ### Define Objective
    ## Objective Function - Total Enjoyment
    solver.Maximize(
        sum(x[(child.name, egg)] * egg_types[egg].enjoyment for child in all_children for egg in egg_names)
    )
    
    ### Define Constraints
    ## Setup constraints per child
    for child in all_children:
        ## Constraint: basket capacity 
        solver.Add(
            sum(x[(child.name, egg)] * egg_types[egg].size for egg in egg_names) <= child.basket_capacity
        )
        
        ## Constraint: Min health level
        solver.Add(
            sum(x[(child.name, egg)] * egg_types[egg].health for egg in egg_names) >= model_parameters.health_min
        )

        ## Constraint: Min happiness level 
        solver.Add(
            sum(x[(child.name, egg)] * egg_types[egg].enjoyment for egg in egg_names) >= model_parameters.happiness_min
        )

        ## Constraint: time limit based on age 
        max_eggs = int(model_parameters.egg_rate * child.effective_time())
        solver.Add(
            sum(x[(child.name, egg)] for egg in egg_names) <= max_eggs
        )
        
        ## Constraint: Each child gets a golden egg
        solver.Add(x[(child.name, model_parameters.golden_egg_name)] == 1)

        ## Constraint: Children's preferences 
        for egg in egg_names:
            if not child.preference.get(egg, False):
                solver.Add(x[(child.name, egg)] == 0)

    ## Setup egg constraints 
    for egg in egg_names:
        ## Constraint: Eggs must not exceed supply 
        solver.Add(
            sum(x[(child.name, egg)] for child in all_children) <= total_supply[egg]
        )

    ## More Complex Constraints: 
    ## Setup fairness metrics for children in houses
    for house in houses:
        child_list = house.children
        for i in range(len(child_list)):
            for j in range(i + 1, len(child_list)):
                ci = child_list[i]
                cj = child_list[j]
                diff = solver.IntVar(-solver.infinity(), solver.infinity(), f'diff_{ci.name}_{cj.name}')
                solver.Add(diff == sum(x[(ci.name, e)] for e in egg_names) - sum(x[(cj.name, e)] for e in egg_names))
                solver.Add(diff <= model_parameters.fairness_threshold)
                solver.Add(diff >= -model_parameters.fairness_threshold)

    ### Solve Problem 
    status = solver.Solve()

    ## Results 
    if status != pywraplp.Solver.OPTIMAL:
        return {"status": "No optimal solution found"}

    result = {}
    for child in all_children:
        result[child.name] = {}
        for egg in egg_names:
            count = int(x[(child.name, egg)].solution_value())
            if count > 0:
                result[child.name][egg] = count
    return result

#### Setup Data

In [55]:
egg_types = {
    "regular": EggType("regular", value=1, enjoyment=2, health=2, size=1, cost=1),
    "golden": EggType("golden", value=5, enjoyment=5, health=1, size=1, cost=3),
    "healthy": EggType("healthy", value=2, enjoyment=1, health=5, size=1, cost=2),
    "chocolate": EggType("chocolate", value=3, enjoyment=4, health=0, size=2, cost=2),
    "white chocolate": EggType("white chocolate", value=3, enjoyment=4, health=0, size=2, cost=2),
    "rum and raisin": EggType("rum and raisin", value=3, enjoyment=2, health=1, size=3, cost=2),
    "dark": EggType("dark", value=3, enjoyment=2, health=1, size=2, cost=2),
}

houses = [
    House(
        id=1,
        children=[
            Child(
                "Alice",
                age=9,
                basket_capacity=10,
                house_id=1,
                preference={
                    "regular": True,
                    "golden": True,
                    "healthy": True,
                    "chocolate": False,
                    "white chocolate": False,
                    "rum and raisin": False,
                    "dark": False,
                },
            ),
            Child(
                "Bob",
                age=12,
                basket_capacity=8,
                house_id=1,
                preference={
                    "regular": True,
                    "golden": True,
                    "healthy": False,
                    "chocolate": True,
                    "white chocolate": False,
                    "rum and raisin": False,
                    "dark": False,
                },
            ),
        ],
    ),
    House(
        id=2,
        children=[
            Child(
                "Timmy",
                age=6,
                basket_capacity=10,
                house_id=2,
                preference={
                    "regular": True,
                    "golden": True,
                    "healthy": True,
                    "chocolate": False,
                    "white chocolate": False,
                    "rum and raisin": False,
                    "dark": False,
                },
            ),
            Child(
                "Bob",
                age=5,
                basket_capacity=12,
                house_id=2,
                preference={
                    "regular": True,
                    "golden": True,
                    "healthy": False,
                    "chocolate": True,
                    "white chocolate": False,
                    "rum and raisin": False,
                    "dark": False,
                },
            ),
        ],
    ),
    House(
        id=3,
        children=[
            Child(
                "Jasmine",
                age=25,
                basket_capacity=20,
                house_id=3,
                preference={
                    "regular": False,
                    "golden": True,
                    "healthy": True,
                    "chocolate": False,
                    "white chocolate": False,
                    "rum and raisin": True,
                    "dark": True,
                },
            ),
            Child(
                "Charlie",
                age=5,
                basket_capacity=12,
                house_id=3,
                preference={
                    "regular": True,
                    "golden": True,
                    "healthy": False,
                    "chocolate": True,
                    "white chocolate": False,
                    "rum and raisin": False,
                    "dark": False,
                },
            ),
        ],
    ),
]

total_supply = {"regular": 10, "golden": 20, "healthy": 50, "chocolate": 20, "white chocolate":10, "rum and raisin":10, "dark":10}

#### Run Optimiser

In [56]:
model_parameters = ModelParameters()

result = optimize_easter_distribution(houses, egg_types, total_supply, model_parameters)


total_cost = 0
total_enjoyment = 0
for house in houses:
      print(f"House number {house.id}")
      for child in house.children:
            print(f"{child.name} got: {result[child.name]}")
            for egg in result[child.name]:
                  total_cost += egg_types[egg].cost
                  total_enjoyment += egg_types[egg].enjoyment

            print()
      print("---\n")

print(f"Total Cost: ${total_cost}")
print(f"Total Enjoyment: {total_enjoyment} 😊 ")

House number 1
Alice got: {'regular': 1, 'golden': 1, 'healthy': 6}

Bob got: {'regular': 3, 'golden': 1, 'chocolate': 2}

---

House number 2
Timmy got: {'golden': 1, 'healthy': 7}

Bob got: {'regular': 3, 'golden': 1, 'chocolate': 2}

---

House number 3
Jasmine got: {'golden': 1, 'dark': 9}

Charlie got: {'regular': 3, 'golden': 1, 'chocolate': 4}

---

Total Cost: $34
Total Enjoyment: 54 😊 


# Output Documentation


In [ ]:
# !pandoc --toc  --standalone --mathjax -f markdown -t docx  ../documentation/model_documentation.md -o ../documentation/model_documentation.docx --reference-doc ../documentation/reference.docx

print("Successfully exported documentation")

Successfully exported documentation
